<a href="https://colab.research.google.com/github/Lossophy/BINA-Projekt/blob/main/import%20git%20-%20Kopie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install -q xlrd
!git clone https://github.com/Lossophy/BINA-Projekt.git

fatal: destination path 'BINA-Projekt' already exists and is not an empty directory.


In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import plotly.express as px
from IPython.display import display
from IPython.display import clear_output
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = 'notebook'  # oder 'iframe' oder 'plotly_mimetype'


In [55]:
df = pd.read_csv('/content/BINA-Projekt/Data/ogd115_gest_bilanz.csv')

In [56]:
df.head()

,Jahr,Rubrik,Energietraeger,TJ
0,1980,Bruttoverbrauch,Elektrizität,-29450.0
1,1980,Bruttoverbrauch,Erdölprodukte,321250.0
2,1980,Bruttoverbrauch,Fernwärme,0.0
3,1980,Bruttoverbrauch,Gas,36280.0
4,1980,Bruttoverbrauch,Holzenergie,26280.0


In [57]:
von_jahr_widget = widgets.Dropdown(
    options=sorted(df['Jahr'].unique()),
    description='Jahr',
    disabled=False,
)

In [58]:
bis_jahr_widget = widgets.Dropdown(
    options=sorted(df['Jahr'].unique()),
    description='Jahr',
    disabled=False,
)

In [59]:
rubrik_widget = widgets.Dropdown(
    options=sorted(df['Rubrik'].unique()),
    value=df['Rubrik'].unique()[0],
    description='Rubrik:'
)


In [60]:
energietraeger_widget = widgets.Dropdown(
    options=sorted(df['Energietraeger'].unique()),
    value=df['Energietraeger'].unique()[0],
    description='energietraeger:'
)


In [61]:
multi_energietraeger_widget = widgets.SelectMultiple(
    options=sorted(df['Energietraeger'].unique()),
    value=tuple(df['Energietraeger'].unique()),
    description='Energie:'
)

In [65]:
# Interaktive Funktion definieren
def bal_plot_daten(jahr, rubrik, energietraeger):
    gefiltert = df[(df['Jahr'] == jahr) &
                   (df['Rubrik'] == rubrik) &
                   (df['Energietraeger'] == energietraeger)]

    if gefiltert.empty:
        print("Keine Daten für diese Kombination.")
        return

    plt.figure(figsize=(6,4))
    plt.bar([0], gefiltert['TJ'], tick_label=[f"{jahr}"])
    plt.ylabel('TJ')
    plt.title(f"{rubrik} - {energietraeger} ({jahr})")
    plt.show()

In [66]:
# Interaktive Ansicht verbinden
ui = widgets.VBox([von_jahr_widget, rubrik_widget, energietraeger_widget])
out = widgets.interactive_output(bal_plot_daten, {
    'jahr': von_jahr_widget,
    'rubrik': rubrik_widget,
    'energietraeger': energietraeger_widget
})

display(ui, out)

Output()

In [67]:
# Plot-Funktion
def plot_daten(von_jahr, bis_jahr, rubrik, energietraeger):
    if von_jahr > bis_jahr:
        print("Fehler: 'Von Jahr' darf nicht größer als 'Bis Jahr' sein.")
        return

    gefiltert = df[(df['Jahr'] >= von_jahr) &
                   (df['Jahr'] <= bis_jahr) &
                   (df['Rubrik'] == rubrik) &
                   (df['Energietraeger'] == energietraeger)]

    if gefiltert.empty:
        print("Keine Daten für diese Auswahl.")
        return

    gruppiert = gefiltert.groupby('Jahr')['TJ'].sum().reset_index()

    plt.figure(figsize=(25,10))
    plt.plot(gruppiert['Jahr'].astype(int), gruppiert['TJ'], marker='o')
    plt.xticks(gruppiert['Jahr'].astype(int))
    plt.xlabel('Jahr')
    plt.ylabel('TJ')
    plt.title(f'{rubrik} - {energietraeger} ({von_jahr}–{bis_jahr})')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Anzeige
ui = widgets.VBox([von_jahr_widget, bis_jahr_widget, rubrik_widget, energietraeger_widget])
out = widgets.interactive_output(plot_daten, {
    'von_jahr': von_jahr_widget,
    'bis_jahr': bis_jahr_widget,
    'rubrik': rubrik_widget,
    'energietraeger': energietraeger_widget
})
clear_output()
display(ui, out)


Output()

In [70]:
# Plotfunktion
def pie_plot(von_jahr, rubrik):
    gefiltert = df[
        (df['Jahr'] == von_jahr) &
        (df['Rubrik'] == rubrik)
    ]

    if gefiltert.empty:
        print("⚠️ Keine Daten verfügbar für diese Auswahl.")
        return

    grouped = gefiltert.groupby('Energietraeger')['TJ'].sum().reset_index()

    fig = go.Figure(data=[go.Pie(
        labels=grouped['Energietraeger'],
        values=grouped['TJ'],
        hoverinfo='label+percent',
        textinfo='label+value'
    )])

    fig.update_layout(
        title=f'Verteilung der Energietraeger – {rubrik} ({von_jahr})',
        height=500
    )

    fig.show()



In [71]:
# Interaktive Verbindung
ui = widgets.HBox([von_jahr_widget, rubrik_widget])
out = widgets.interactive_output(pie_plot, {
    'von_jahr': von_jahr_widget,
    'rubrik': rubrik_widget
})

display(ui, out)

Output()